In [2]:
# build a keras CNN model
# from http://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

num_classes = 3
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=[75, 75, 1]))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])


Using TensorFlow backend.


In [8]:
# useless
import matplotlib.pyplot as plt
import scipy.ndimage
shapes = []
for i in range(1, 3001):
    img = scipy.ndimage.imread('/Users/luoxin/Desktop/training_set/images/img_%04d.jpg' % i)
    shapes.append(img.shape)
    print(i)

zsize = list(zip(*shapes))
print(len(zsize))
print(min(zsize[0]))  # 103
print(min(zsize[1]))  # 76

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073


(75, 75, 3)


In [10]:
# test code: downsampling images
import scipy.misc
img = scipy.ndimage.imread('/Users/luoxin/Desktop/training_set/images/img_0001.jpg')
img2 = scipy.misc.imresize(img, [75, 75])
plt.imshow(img2)
plt.show()
print(img2.shape)

In [14]:
# read labels from file
import csv
import numpy as np
with open('/Users/luoxin/Desktop/training_set/label_train.csv', newline='') as f:
    dataset = csv.reader(f, delimiter=',')
    dataset.__next__()
    zdset = list(zip(*dataset))

labels = zdset[1]
one_hots = np.zeros((len(labels), 3))

for k in range(len(labels)):
    one_hots[k, int(labels[k])] = 1

In [16]:
# prepare the images

import numpy as np

def x_train_gen():
    for i in range(1, 3001):
        img = scipy.ndimage.imread('/Users/luoxin/Desktop/training_set/images/img_%04d.jpg' % i)
        yield scipy.misc.imresize(img, [75, 75])

        
all_data_set = []
for k, img in enumerate(x_train_gen()):
    all_data_set.append(img)

In [19]:
# preprocess the images into gray scale

def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray


images = []
for x in all_data_set:
    if len(x.shape) == 3:
        y = rgb2gray(x)
    else:
        y = x
    y = np.expand_dims(y, 2)
    images.append(y)

print(len(images))
print(images[0].shape)

3000
(75, 75, 1)


In [58]:
# partition of training and validation dataset

N = 2250
training_x = images[:N]
training_y = one_hots[:N, :]

validation_x = images[N:]
validation_y = one_hots[N:, :]

In [59]:
# training

batch_size = 20
epochs = 20
cn_fit=model.fit(np.array(training_x), np.array(training_y),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(validation_x), np.array(validation_y)))

Train on 2250 samples, validate on 750 samples
Epoch 1/20
2250/2250 [==============================] - 133s - loss: 10.7884 - acc: 0.3307 - val_loss: 10.6165 - val_acc: 0.3413
Epoch 2/20
2250/2250 [==============================] - 123s - loss: 10.7884 - acc: 0.3307 - val_loss: 10.6165 - val_acc: 0.3413
Epoch 3/20
2250/2250 [==============================] - 114s - loss: 10.7884 - acc: 0.3307 - val_loss: 10.6165 - val_acc: 0.3413
Epoch 4/20
2250/2250 [==============================] - 105s - loss: 10.7884 - acc: 0.3307 - val_loss: 10.6165 - val_acc: 0.3413
Epoch 5/20
2250/2250 [==============================] - 86s - loss: 10.7884 - acc: 0.3307 - val_loss: 10.6165 - val_acc: 0.3413
Epoch 6/20
2250/2250 [==============================] - 73s - loss: 10.7884 - acc: 0.3307 - val_loss: 10.6165 - val_acc: 0.3413
Epoch 7/20
2250/2250 [==============================] - 82s - loss: 10.7884 - acc: 0.3307 - val_loss: 10.6165 - val_acc: 0.3413
Epoch 8/20
2250/2250 [==============================]

In [79]:
result=[]
for i in range(100):
    #print(i)
    scores = model.predict(validation_x[i].reshape(1,75,75,1))
    result.extend(scores.tolist())

TypeError: 'Tensor' object is not callable

In [80]:
result

[[1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0.0

In [81]:
np.mean(np.array(result)!=validation_y[1])

0.0

In [88]:
training_y

array([[ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       ..., 
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.]])

In [82]:
np.array(result)

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.